In [1]:
from IPython.display import display

from super_gradients.training import models
from super_gradients.training import Trainer

from torchinfo import summary

from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

The console stream is logged into /home/aws_install/sg_logs/console.log


[2023-05-07 23:08:35] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-05-07 23:08:52] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command


I0507 23:08:52.764481 140572416942272 env_sanity_check.py:84] Library check is not supported when super_gradients installed through "git+https://github.com/..." command


In [2]:
model = models.get("yolo_nas_s", pretrained_weights="coco") # s, m or l

summary(model=model, 
        input_size=(16, 3, 2048, 2048),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='lard_yolo_nas_s', ckpt_root_dir=CHECKPOINT_DIR)

Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /home/aws_install/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth


  0%|          | 0.00/73.1M [00:00<?, ?B/s]

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
# LARD

# TODO: Update parameters
dataset_params = {
    # 'data_dir':'/content/U.S.-Coins-Dataset---A.Tatham-5',
    'data-dir': '/aip/datasets/a3/preprocessed/07-02-23/yolo-formated/detect',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    # 'test_images_dir':'test/images',
    'test_images_dir':'valid/images',
    # 'test_labels_dir':'test/labels',
    'test_labels_dir':'valid/labels',
    'classes': ['Runway']
}

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':8
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':8
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':6
    }
)


display(train_data.dataset.transforms)

# Plot a batch
train_data.dataset.plot()